In [17]:
from dotenv import load_dotenv
from openai import OpenAI
from time import sleep
from os import listdir

In [4]:
# load environment variables (API keys)
# should be in .env file OPENAI_API_KEY=sk-...
load_dotenv()

client = OpenAI()


In [20]:
# Uplaod file should only be done once

# file = client.files.create(
#     file=open(file_path, "rb"),
#     purpose='assistants'
# )

document_path = "documents"
# print all file names in the documents folder
for document in listdir(document_path):
    print("Uploading file: ", document)
    file = client.files.create(
        file=open(f"{document_path}/{document}", "rb"),
        purpose='assistants'
    )

    # append file id to new file
    print("Writing file id to file_ids.txt")
    with open("file_ids.txt", "a") as f:
        f.write(file.id + "\n")





Uploading file:  Förord.pdf
Writing file id to file_ids.txt
Uploading file:  Svensson.pdf
Writing file id to file_ids.txt
Uploading file:  Larsson2.pdf
Writing file id to file_ids.txt
Uploading file:  Otterbeck.pdf
Writing file id to file_ids.txt
Uploading file:  Olsson & Sorgenfrei.pdf
Writing file id to file_ids.txt
Uploading file:  Sorgenfrei Olssson.pdf
Writing file id to file_ids.txt
Uploading file:  Hjärpe.pdf
Writing file id to file_ids.txt
Uploading file:  Olsson2.pdf
Writing file id to file_ids.txt
Uploading file:  Sorgenfrei.pdf
Writing file id to file_ids.txt
Uploading file:  Poljarevic2.pdf
Writing file id to file_ids.txt
Uploading file:  Stenberg.pdf
Writing file id to file_ids.txt
Uploading file:  Olsson1.pdf
Writing file id to file_ids.txt
Uploading file:  Larsson.pdf
Writing file id to file_ids.txt
Uploading file:  Thurfjell.pdf
Writing file id to file_ids.txt
Uploading file:  Sorgenfrei & Olsson.pdf
Writing file id to file_ids.txt
Uploading file:  Larsson, Olsson Sor

In [25]:
# files = []

# for line in open("file_ids.txt", "r").readlines():
#     files.append(line.strip())

print("Files: ", files)


Files:  ['file-6ZccOWyCX4eQ7NBJuNJf5fCI', 'file-uIMf2TH9QcHVm8Dq8gaWSmiz', 'file-8dAXXZ9kFZPsShdmd5rJmSBN', 'file-jwevSdA3NsERT8qb2cyFRcPa', 'file-g5FadGJzTx5TrpymSlTe1O5N', 'file-AdlVpSZeyVNGK2sLcpzRf5o2', 'file-JH2S1YLQ4P6N5uA6nNxoIQQI', 'file-jdPlomchPXU8AHS9QPt3qgrK', 'file-sCoQ9OgcmT6r0YDdalwinNlq', 'file-82hyGrdoVLvxp6kYpNNOQSY6', 'file-RD2QBVJpa2jWgdyBfMcrbclh', 'file-UBhKtSB4FpB4IKSXnITumORf', 'file-hRStoBY3LiKV0hEzXN1WBinm', 'file-p1mdAwmS49rw9DmpZVPdC549', 'file-1BwubI6X50DA0sr20gv1GKHR', 'file-1LRr6FEOoDjWMpZsBpgFWmdV', 'file-jFU0wXh2wTbaB6dk6NbtkWjC']


In [27]:

'''
This is the assistant that will be used to answer questions about the book.

the assistant is only create once and can be used multiple times. with assitant.id
'''
assistant = client.beta.assistants.create(
    name="Islam quiz creator",
    instructions="""
    You are a quizz creator for creating quizzes to students as study material.
    Do not give paragraphs but instead only questions.
    Create a quiz about Islam from the provided text.
    Use only the provided text and craete a generall quizz to help people learn about Islam.
    If the answer cannot be found in the articles, write 'I could not find an answer from the provded documents.'""",
    #
    # retrieval tool is used to implement a RAG-system
    # 
    tools=[{"type": "retrieval"}],
    model="gpt-3.5-turbo-1106",
    file_ids=files
)

print(assistant.id)

asst_DsTDGQ1pD9gYm9Lehy9VSgu4


In [29]:

# client.files.list()

my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)

for assistangt in my_assistants.data:
    # print("delting assistant: ", assistangt.id)
    #client.beta.assistants.delete(assistangt.id)

    print(assistangt)

# my_assistants.data




Assistant(id='asst_DsTDGQ1pD9gYm9Lehy9VSgu4', created_at=1703330997, description=None, file_ids=['file-jFU0wXh2wTbaB6dk6NbtkWjC', 'file-1LRr6FEOoDjWMpZsBpgFWmdV', 'file-1BwubI6X50DA0sr20gv1GKHR'], instructions="\n    You are a quizz creator for creating quizzes to students as study material.\n    Do not give paragraphs but instead only questions.\n    Create a quiz about Islam from the provided text.\n    Use only the provided text and craete a generall quizz to help people learn about Islam.\n    If the answer cannot be found in the articles, write 'I could not find an answer from the provded documents.'", metadata={}, model='gpt-3.5-turbo-1106', name='Islam quiz creator', object='assistant', tools=[ToolRetrieval(type='retrieval')])
Assistant(id='asst_ScGfhT0u4MomilSUCTuKnRWF', created_at=1703330932, description=None, file_ids=['file-jFU0wXh2wTbaB6dk6NbtkWjC', 'file-1LRr6FEOoDjWMpZsBpgFWmdV', 'file-1BwubI6X50DA0sr20gv1GKHR', 'file-p1mdAwmS49rw9DmpZVPdC549', 'file-hRStoBY3LiKV0hEzXN1WB

In [ ]:



thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Give me 5 quizz questions about Islam. from the provided text only."
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistantID,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)


while run.status != "completed":
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    sleep(1)

print(run.status)



In [ ]:

# retrieve and format mesasge 
messages = client.beta.threads.messages.list(thread_id=thread.id)
message = messages.data[0]

# Extract the message content
message_content = message.content[0].text
annotations = message_content.annotations
citations = []

# Iterate over the annotations and add footnotes
for index, annotation in enumerate(annotations):
    # Replace the text with a footnote
    message_content.value = message_content.value.replace(
        annotation.text, f' [{index}]')

    # Gather citations based on annotation attributes
    if (file_citation := getattr(annotation, 'file_citation', None)):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(
            f'[{index}] {file_citation.quote} from {cited_file.filename}')
    elif (file_path := getattr(annotation, 'file_path', None)):
        cited_file = client.files.retrieve(file_path.file_id)
        citations.append(
            f'[{index}] Click <here> to download {cited_file.filename}')
        # Note: File download functionality not implemented above for brevity

# Add footnotes to the end of the message before displaying to user
message_content.value += '\n' + '\n'.join(citations)


In [ ]:
print(message_content.value)